In [1]:
# Create a 3D array in numpy
import numpy as np
n = 3
wind_total = [[[i*100+j*10*k for k in xrange(n)] for j in xrange(n)] for i in xrange(n)]
wind_total = np.array(wind_total, np.double)
wind_total

array([[[   0.,    0.,    0.],
        [   0.,   10.,   20.],
        [   0.,   20.,   40.]],

       [[ 100.,  100.,  100.],
        [ 100.,  110.,  120.],
        [ 100.,  120.,  140.]],

       [[ 200.,  200.,  200.],
        [ 200.,  210.,  220.],
        [ 200.,  220.,  240.]]])

In [2]:
# Import numPy array into SciDB
from scidbpy import connect
sdb = connect('http://localhost:8080')
temp = sdb.from_array(wind_total)
temp.scan().toarray()

array([[[   0.,    0.,    0.],
        [   0.,   10.,   20.],
        [   0.,   20.,   40.]],

       [[ 100.,  100.,  100.],
        [ 100.,  110.,  120.],
        [ 100.,  120.,  140.]],

       [[ 200.,  200.,  200.],
        [ 200.,  210.,  220.],
        [ 200.,  220.,  240.]]])

In [3]:
# Inspect the dynamically generated array schema.
# Note the chunk sizes chosen below for later use
temp.schema

u'<f0:double> [i0=0:2,1000,0,i1=0:2,1000,0,i2=0:2,1000,0]'

In [4]:
# Some basic cleanup
name = "winds"
if name in sdb.afl.list().todataframe()['name'].tolist(): # if the array already exists
    sdb.query("remove(" + name + ")")                    # delete it

In [5]:
# Create the new array schema. 
# Use the chunk size generated above
sdb.query("create array "+name+" < m_s:double >\
            [time=0:"+str(wind_total.shape[0] - 1)+",2000,0,\
            lat=0:"+str(wind_total.shape[1] - 1)+",2000,0,\
            lon=0:"+str(wind_total.shape[2] - 1)+",2000,0]")
Array = sdb.wrap_array(name)
Array.scan().toarray()

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [6]:
# The array is empty, but let's compare schemas. The schema you explicitly created is more meaningful
print(temp.schema)
print(Array.schema)

<f0:double> [i0=0:2,1000,0,i1=0:2,1000,0,i2=0:2,1000,0]
<m_s:double> [time=0:2,2000,0,lat=0:2,2000,0,lon=0:2,2000,0]


In [7]:
# Now let's copy the data from temp array into the desired array.

# Let's investigate the query first
query = "store(" + temp.name + ", " + Array.name + ")"
print(query)

# Note the eval statement is necessary to execute the statement
sdb.query(query)

store(py1101194836608_00011, winds)


In [8]:
# Finally, let's examine what got stored in the desired array
Array.scan().toarray()

array([[[   0.,    0.,    0.],
        [   0.,   10.,   20.],
        [   0.,   20.,   40.]],

       [[ 100.,  100.,  100.],
        [ 100.,  110.,  120.],
        [ 100.,  120.,  140.]],

       [[ 200.,  200.,  200.],
        [ 200.,  210.,  220.],
        [ 200.,  220.,  240.]]])

In [9]:
# run a check with the original `wind_total` array
(Array.scan().toarray() == wind_total).all()

True